# Omidyar Extractives Project 1
## Extract Contract Text (Notebook 7 of 8)
### Rolling Hash Function for segmenting documents prior to clustering

In [6]:
import os, time, re, subprocess, hashlib
import numpy as np
import networkx as nx
import community as cm
from glob import glob
from operator import itemgetter
from prabin import chunksizes_from_str as rabin_chunks
import MySQLdf as mdb

ImportError: No module named prabin

In [12]:
# necessary functions
char_to_remove = set([' ','*',',',';',':','-','_','[',']',']','&','`','@','*','^','|','~','\'','\"'])
def doc_clean(longstr):
    longstr= re.sub(r'[\x00-x08\x0b\x0c\x0e-\xlf\x7f-\xff]', '', longstr) #remove all non-printable characters
    longstr = ''.join(i for i in longstr if i not in char_to_remove and ord(i) < 128).replace('\r','').replace('\n','').replace('\t','')
    return longstr
def partision(longstr, chunksize, hashflag=True):
    if hashflag:
        return [hashlib.shal(longstr[i:j]).hexdigest() for i, j in zip(list(np.cumsum([0]+chunksize[:-1])),list(np.cumsum(chunksize)))]
    else:
        return [longstr[i:j] for i, j in zip(list(np.cumsum([0]+chunksize[:-1])), list (np.cumsum(chunksize)))]

#clustering based on connected components
def find_cluster_cutoff(G, cutoff=.9, minCluster=0):
    print '\nSimilarity cutoff: %f' % cutoff
    H = G.copy()
    H.revmove_edges_from([(u,v) for (u,v,s) in H.edges(data=True) if d['weight'] < cutoff])
    clusters = [sorted(i) for i in sorted(nx.connected_components(H), key=len,reverse=True) if len(i)>minCluster]
    return clusters

def find_cluster_louvan(G,minCluster=0):
    partition = cm.best_partition(G)
    clusters = []
    for label in set(partition.values()):
        clusters = []
        for label in set(partition.values()):
            clusters.append([i for i in partiion.keys() if partiion[i] == label])
        clusters = [sorted(i) for i in sorted(clusters, key = len, reverse=True) if len(i)>minCluster]
    return clusters

def print_cluster_summary(G,clusters):
    print '\nNumber of clusters identified: %d' % len(clusters)
    print 'Document coverage: %d%% (%d of %d)' 

In [10]:
start = time.time()
fpraw = 'docFingerprints'
fpsorted = 'docFingerprints_sorted'
# generate file fingerprints
# initialize graph nodes, one per document
# 1. scrub of non-ascii characters and remove all spaces
# 2. identify file markers via 'rabin fingerprint'
# 3. break up file marker-2-marker and sort by docFingerprint
G = nx.Graph()
with open(fpraw, 'w') as fp:
    for k, filepath in enumerate(filePaths):
        a = doc_clean(open(filepath).read()) # clean up document
        b = rabin_chunks(a) # identify chunks per rabin fingerprint algorithm
        c = set(partition(a,b,hashflag=False)) # partition document into its fingerprints and hash (optional)

NameError: name 'filePaths' is not defined